In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [11]:
# Ensure TensorFlow is using GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available and enabled.")
else:
    print("No GPU found. Running on CPU.")

GPU is available and enabled.


In [12]:
# Define dataset directory
dataset_dir = "dataset"
img_size = (224, 224)
batch_size = 32
n_splits = 5
random_state = 42

In [13]:
# Function to load all images and labels
def load_dataset(dataset_path):
    images = []
    labels = []
    classes = os.listdir(dataset_path)

    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        print(f"Loading class: {class_name}")

        for img_name in tqdm(os.listdir(class_path)):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(class_idx)

    return np.array(images), np.array(labels), classes

X, y, class_names = load_dataset(dataset_dir)
print(f"Dataset loaded: {X.shape[0]} images across {len(class_names)} classes")

# Initial train/test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=random_state, stratify=y
)

# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation/test data
test_datagen = ImageDataGenerator(rescale=1.0/255)


Loading class: Arjantin


100%|██████████| 21/21 [00:00<00:00, 248.89it/s]


Loading class: Buchkeygala


100%|██████████| 21/21 [00:00<00:00, 135.48it/s]


Loading class: Galaval


...

Loading class: Joremin


100%|██████████| 21/21 [00:00<00:00, 202.13it/s]


Loading class: Superchief


100%|██████████| 21/21 [00:00<00:00, 201.93it/s]

Dataset loaded: 120 images across 6 classes


In [15]:
def create_model(base_model_name='resnet50'):
    if base_model_name.lower() == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*img_size, 3))
    elif base_model_name.lower() == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*img_size, 3))
    else:
        raise ValueError("Supported models: 'resnet50' or 'vgg16'")

    base_model.trainable = False
    global_avg_pool = GlobalAveragePooling2D()(base_model.output)
    output_layer = Dense(len(class_names), activation='softmax')(global_avg_pool)
    model = Model(inputs=base_model.input, outputs=output_layer)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [16]:
def perform_cross_validation(model_name, X_data, y_data, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_data)):
        print(f"\nTraining {model_name} - Fold {fold+1}/{n_splits}")

        # Split data for this fold
        X_fold_train, X_fold_val = X_data[train_idx], X_data[val_idx]
        y_fold_train, y_fold_val = y_data[train_idx], y_data[val_idx]

        # Create and train model
        model = create_model(model_name)

        # Create data generators for this fold
        train_generator = train_datagen.flow(
            X_fold_train, y_fold_train, batch_size=batch_size
        )

        val_generator = test_datagen.flow(
            X_fold_val, y_fold_val, batch_size=batch_size
        )

        # Train model
        history = model.fit(
            train_generator,
            epochs=10,  # Reduced for example
            validation_data=val_generator,
            verbose=1
        )

        # Evaluate on validation set
        val_loss, val_acc = model.evaluate(
            X_fold_val / 255.0, y_fold_val, verbose=0
        )

        fold_accuracies.append(val_acc)
        print(f"{model_name} - Fold {fold+1} Validation Accuracy: {val_acc:.4f}")

    # Overall performance
    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    print(f"\n{model_name} - Mean Accuracy: {mean_acc:.4f} (±{std_acc:.4f})")

    return fold_accuracies, mean_acc, std_acc

In [19]:
# Run cross-validation for ResNet50
resnet_results = perform_cross_validation('resnet50', X_train, y_train, n_splits=n_splits)


Training resnet50 - Fold 1/5
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1443 - loss: 2.0532 - val_accuracy: 0.2000 - val_loss: 1.7887
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.1571 - loss: 1.8264 - val_accuracy: 0.1000 - val_loss: 1.8165
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - accuracy: 0.1308 - loss: 1.7942 - val_accuracy: 0.1000 - val_loss: 1.8748
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.1665 - loss: 1.8370 - val_accuracy: 0.1000 - val_loss: 1.9124
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.2054 - loss: 1.7971 - val_accuracy: 0.1000 - val_loss: 1.9058
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.1859 - loss: 1.8302 - val_accuracy: 0.1000 - val_loss: 1.8536
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.2057 - loss: 1.7844 - val_accuracy: 0.1500 - val_loss: 1.8125
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.1681 - loss: 1.7891 - val_

In [18]:
# Run cross-validation for VGG16
vgg_results = perform_cross_validation('vgg16', X_train, y_train, n_splits=n_splits)


Training vgg16 - Fold 1/5
Epoch 1/10


2025-04-04 14:09:23.183754: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_573', 84 bytes spill stores, 84 bytes spill loads

2025-04-04 14:09:25.024482: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1485 - loss: 2.1537

2025-04-04 14:09:39.753276: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 84 bytes spill stores, 84 bytes spill loads

2025-04-04 14:09:39.799048: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 12 bytes spill stores, 12 bytes spill loads

2025-04-04 14:09:40.822442: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3/3 ━━━━━━━━━━━━━━━━━━━━ 25s 7s/step - accuracy: 0.1574 - loss: 2.1401 - val_accuracy: 0.1000 - val_loss: 2.1419
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - accuracy: 0.1979 - loss: 1.9764 - val_accuracy: 0.1000 - val_loss: 2.0186
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.1631 - loss: 1.9667 - val_accuracy: 0.1000 - val_loss: 1.9309
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 207ms/step - accuracy: 0.1574 - loss: 1.8830 - val_accuracy: 0.1000 - val_loss: 1.8738
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 205ms/step - accuracy: 0.2159 - loss: 1.8241 - val_accuracy: 0.1000 - val_loss: 1.8380
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step - accuracy: 0.2445 - loss: 1.7630 - val_accuracy: 0.1500 - val_loss: 1.8160
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step - accuracy: 0.2368 - loss: 1.7860 - val_accuracy: 0.1000 - val_loss: 1.8047
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 204ms/step - accuracy: 0.1640 - loss: 1.7686 - val_accuracy: 0.1000 - val_loss: 1.7913
Epoch

2025-04-04 14:10:01.605870: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 84 bytes spill stores, 84 bytes spill loads

2025-04-04 14:10:01.674678: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_199', 12 bytes spill stores, 12 bytes spill loads

2025-04-04 14:10:02.492874: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.2067 - loss: 1.9863 - val_accuracy: 0.1053 - val_loss: 2.1594
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.1490 - loss: 2.0043 - val_accuracy: 0.1053 - val_loss: 2.0486
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 258ms/step - accuracy: 0.1683 - loss: 1.9264 - val_accuracy: 0.1053 - val_loss: 1.9508
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.1912 - loss: 1.8695 - val_accuracy: 0.1053 - val_loss: 1.8853
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step - accuracy: 0.2233 - loss: 1.7743 - val_accuracy: 0.0526 - val_loss: 1.8432
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.2672 - loss: 1.7488 - val_accuracy: 0.1579 - val_loss: 1.8264
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.2304 - loss: 1.7666 - val_accuracy: 0.2105 - val_loss: 1.8233
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - accuracy: 0.1662 - loss: 1.7697 - val_accuracy: 0.1053 - val_loss: 1.8237
Epoch

In [14]:
# Feature Extraction using SURF or ORB
def extract_features(img_dir, use_surf=True):
    feature_extractor = cv2.xfeatures2d.SURF_create() if use_surf else cv2.ORB_create()
    features, labels = [], []
    class_labels = os.listdir(img_dir)
    for label in class_labels:
        class_path = os.path.join(img_dir, label)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (224, 224))
            keypoints, descriptors = feature_extractor.detectAndCompute(img, None)
            if descriptors is not None:
                features.append(descriptors.flatten())
                labels.append(label)
    return np.array(features, dtype=object), np.array(labels)
            if descriptors is not None:
                features.append(descriptors.flatten())
                labels.append(label)

    return np.array(features, dtype=object), np.array(labels)

In [15]:
X, y = extract_features(dataset_dir, use_surf=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
models = {
    "SVM": SVC(kernel='linear'),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

for name, clf in models.items():
    scores = cross_val_score(clf, X_train, y_train, cv=5)
    print(f"{name} Accuracy: {scores.mean():.4f}")

SVM Accuracy: 0.2216
KNN Accuracy: 0.1690
Decision Tree Accuracy: 0.1269
Random Forest Accuracy: 0.1895
